In [1]:
from tqdm import tqdm

# bag of wordsの辞書を作成
def create_BoW_dict(file_path):
    BoW_dict = {}
    i = 0
    with open(file_path, "r", encoding="utf-8")as f:
        data_list = f.readlines()
        for data in data_list:
            data = data.split('\t')
            word_list = data[0].split()
            for word in word_list:
                if word in BoW_dict:
                    continue
                else:
                    BoW_dict[word] = i
                    i += 1
    return BoW_dict

# t=0, e=1, m=2, b=3としてテストデータのベクトルとラベルをそれぞれリストとして作成
def create_vector_and_label(file_path, BoW_dict):
    with open(file_path, "r", encoding="utf-8")as f:
        data_list = f.readlines()
        vector_list = []
        label_list = []
        for data in tqdm(data_list):
            data = data.split("\t")
            word_list = data[0].split()
            vector = [0] * len(BoW_dict)
            for word in word_list:
                if word in BoW_dict:
                    vector[BoW_dict[word]] += 1
            vector_list.append(vector)
            label_alph = data[1].strip("\n")
            if label_alph == "t":
                label = [1, 0, 0, 0]
            elif label_alph == "e":
                label = [0, 1, 0, 0]
            elif label_alph == "m":
                label = [0, 0, 1, 0]
            elif label_alph == "b":
                label = [0, 0, 0, 1]
            else:
                print("error label")
            label_list.append(label)
    return vector_list, label_list


TRAIN_FEATURE_FILE_PATH = "./news+aggregator/train.feature.txt"

# bag of wordsの辞書型を作成
BoW_dict = create_BoW_dict(TRAIN_FEATURE_FILE_PATH)

In [2]:
import torch

# モデルのクラスが定義されていないとloadできないため
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.l1 = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.l1(x)
        return x

# 52で学習したモデルのロード
model = torch.load('model_weight.pth')

In [3]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

correct = 0
total = 0

TEST_FEATURE_FILE_PATH = "./news+aggregator/test.feature.txt"

# test.txtからベクトルリストとラベルリストを作成
x_test_list, y_test_list = create_vector_and_label(TEST_FEATURE_FILE_PATH, BoW_dict)

# データの準備
x_test = torch.Tensor(x_test_list)
y_test = torch.Tensor(y_test_list).long()

# 訓練データのデータローダー
test = TensorDataset(x_test, y_test)
test_loader = DataLoader(test, batch_size=5)

#　testデータの各ラベルの確立を出力
for x, y in test_loader:
    outputs = model(x)
    print(outputs)


100%|██████████| 1334/1334 [00:00<00:00, 7706.53it/s]


tensor([[-1.5558,  1.4299,  0.2531, -0.0987],
        [-1.2679,  3.1212, -1.2792, -0.5909],
        [-1.3485,  1.6782, -0.8515,  0.5580],
        [ 0.2025, -0.2886,  0.6024, -0.4319],
        [ 0.4323, -0.3969, -0.6837,  0.6565]], grad_fn=<AddmmBackward0>)
tensor([[ 1.4473, -1.2430, -2.5280,  2.3003],
        [-1.7280, -1.7196, -1.3381,  4.8164],
        [ 0.6280, -3.2358, -1.1484,  3.7631],
        [ 0.0377,  0.5206, -2.1736,  1.6227],
        [-0.3067,  1.0222, -0.9689,  0.2946]], grad_fn=<AddmmBackward0>)
tensor([[-2.4668, -0.3840, -1.6916,  4.5542],
        [ 0.2068, -1.2634, -1.5860,  2.6335],
        [-1.2082,  2.7805, -1.2148, -0.3309],
        [-1.2116, -2.7839, -1.9978,  6.0053],
        [-1.3522,  0.1581, -1.4956,  2.7332]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0811, -1.2722,  0.1410,  1.0421],
        [ 0.6079, -2.0268, -1.4885,  2.9435],
        [-1.9778,  6.4987, -1.6910, -2.7797],
        [-0.7052,  1.7463, -1.1414,  0.1429],
        [ 0.9371, -2.4202,  1.2270,  0.2607]]